In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
from scipy import ndimage
from tqdm import tqdm

In [ ]:
def clear_layers(viewer):
    layers = viewer.layers
    while len(layers) > 0:
        layers.remove(layers[0])

# load the cellpose segmentation file

From the docs (https://cellpose.readthedocs.io/_/downloads/en/latest/pdf/) segmentations is a dictionary object with the following fields:
 - filename : filename of image
 - img : image with chosen channels (nchan x Ly x Lx) (if not multiplane)
 - masks : each pixel in the image is assigned to an ROI (0 = NO ROI; 1,2,. . . = ROI labels)
 - colors : colors for ROIs
 - outlines : outlines of ROIs (0 = NO outline; 1,2,. . . = outline labels)
 - chan_choose : channels that you chose in GUI (0=gray/none, 1=red, 2=green, 3=blue)
 - ismanual : element k = whether or not mask k was manually drawn or computed by the cellpose algorithm
 - flows [flows[0] is XY flow in RGB, flows[1] is the cell probability in range 0-255 instead of 0.0 to 1.0, flows[2]
is Z flow in range 0-255 (if it exists, otherwise zeros),] flows[3] is [dY, dX, cellprob] (or [dZ, dY, dX,
cellprob] for 3D), flows[4] is pixel destinations (for internal use)
 - est_diam : estimated diameter (if run on command line)
 - zdraw : for each mask, which planes were manually labelled (planes in between manually drawn have interpolated
ROIs)

In [ ]:
import numpy as np
import napari

segmenations_path = './220624_Fix_Flvw_Emb_PI_Utr647_E02-10X-Z01_downsample_log_seg-cyto.npy'
segmentations = np.load(segmenations_path, allow_pickle=True).item()

# Generate random XY, ZX, and ZY slices from a training dataset

Useful for generating training data for a 3D cellpose model.

In [ ]:
import numpy as np
from numpy.random import randint
from tifffile import imread, imwrite
from tqdm import tqdm
import os

def get_rand_slices(im: np.ndarray, samples = 3):
    ''' 
    Returns a dictionary of random XY, XZ, and YZ slices from a 3D stack.
    Parameters: im - a 3D stack
                samples - the number of slices for each dimension to return
    Returns:    a dictionary of slices labeled with their dimension and index
    '''
    assert im.ndim == 3, 'Image must be 3D'
    d = {}
    z,y,x = im.shape
    for n in range(samples):
        randyx = randint(0,z)
        randzx = randint(0,y)
        randzy = randint(0,x)
        d[f"randyx_{randyx}_#{n}"]=im[randyx]
        d[f"randzx_{randzx}_#{n}"]=im[:,randzx]
        d[f"randzy_{randzy}_#{n}"]=im[:,:,randzy]
    return d

source = '/Volumes/bigData/wholeMount_volDist/220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Processed/N2V_Denoised/16bit_scaleZ_sbdl2'
dest = '/Volumes/bigData/wholeMount_volDist/220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Processed/N2V_Denoised/16bit_scaleZ_sbdl2_random_slices'
if not os.path.exists(dest):
    os.mkdir(dest)
names = [f for f in os.listdir(source) if f.endswith('_16bit_scaleZ_sbdl2_16bit.tif') and not f.startswith('.')]
for name in tqdm(names):
    im = imread(source +'/'+name)
    justname = name.split('.')[0]
    slices = get_rand_slices(im, samples = 3)
    for k,v in slices.items():
        imwrite(f'{dest}/{justname}_{k}.tif',v)

# Augment dataset by artificially turning xy slices into xz slices by adding 1D Gaussian Blur

#### Example of synthetic training data:

In [ ]:
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
from scipy import ndimage

og_path = ''
og = imread(og_path)
# a Gaussian filter with a standard deviation of 10
gauss = ndimage.gaussian_filter1d(og, 7, 0)

fig, (ax1, ax2) = plt.subplots(1, 2, dpi=200)
ax1.imshow(og, cmap='gray', vmin = 0, vmax=15000)
ax2.imshow(gauss, cmap='gray', vmin = 0, vmax=15000)

#### Batch processing:

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
from scipy import ndimage
from tqdm import tqdm


target_dir = ''
ims = [f for f in os.listdir(target_dir) if f.endswith('tif') and 'yx' in f and not f.startswith('.')]
segs = [f for f in os.listdir(target_dir) if f.endswith('.npy') and not f.startswith('.')]

# gaussian blur the images:
with tqdm(total = len(ims)) as pbar:
    for im in ims:
        im_path = os.path.join(target_dir, im)
        im_data = imread(im_path)
        im_data = ndimage.gaussian_filter1d(im_data, 7.5, 0)
        imwrite(im_path, im_data)
        pbar.update(1)
    
# rename the images and segmentation files:
for f in ims:
    base = f.split('.')[0]
    newname = base + '_gauss.tif'
    os.rename(os.path.join(target_dir, f), os.path.join(target_dir, newname))

for s in segs:
    base = s.split('_seg.npy')[0]
    newname = base + '_gauss_seg.npy'
    os.rename(os.path.join(target_dir, s), os.path.join(target_dir, newname))

# Convert .npy files to labels and save as tif files

This is necessary for training cellpose in GDrive

In [ ]:
from tifffile import imwrite
import numpy as np
from tqdm import tqdm
import shutil
import os

# dir containing ...tif and ..._seg.npy files
raw_dir = '/Volumes/bigData/wholeMount_volDist/CellposeTraining_V2' 
tifs = [f for f in os.listdir(raw_dir) if f.endswith('.tif') and not f.startswith('.')]
segs = [f for f in os.listdir(raw_dir) if f.endswith('.npy') and not f.startswith('.')]

# create save directories
base_save_dir = '/Volumes/bigData/wholeMount_volDist/CellposeTraining_V2_GD'
image_save_dir = os.path.join(base_save_dir, 'Images')
mask_save_dir = os.path.join(base_save_dir, 'Masks')
if not os.path.exists(image_save_dir):
    os.makedirs(image_save_dir)
if not os.path.exists(mask_save_dir):
    os.makedirs(mask_save_dir)

# copy all tifs to the image save dir
for f in tifs:
    shutil.copy(os.path.join(raw_dir, f), image_save_dir)

# extract the masks out of each of the seg files and save as tif to the mask save dir
with tqdm(total=len(segs)) as pbar:
    for s in segs:
        item = np.load(os.path.join(raw_dir, s), allow_pickle=True).item()
        masks = item['masks']
        save_name = s.split('_seg.npy')[0] + '.tif'
        imwrite(os.path.join(mask_save_dir, save_name), masks)
        pbar.update(1)

Convert a single npy file into masks

In [ ]:
from tifffile import imwrite
import numpy as np
import os

# dir containing ...tif and ..._seg.npy files
dir = '/Volumes/bigData/wholeMount_volDist/220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Processed/N2V_Denoised/16bit_scaleZ_sbdl2'
name = '220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Cntrl_E02-Z01_PI_16bit_scaleZ_sbdl2_16bit_seg.npy' 

# create save directories
item = np.load(os.path.join(dir, name), allow_pickle=True).item()
masks = item['masks']
save_name = name.split('_seg.npy')[0] + '_Masks_Fused.tif'
imwrite(os.path.join(dir, save_name), masks, imagej=True, metadata={'axes': 'ZYX'})

# Batch process volumes with DoG filters

In [ ]:
import os
from tqdm import tqdm
from skimage import filters
from tifffile import imread, imwrite

processing_dir = '/Volumes/bigData/wholeMount_volDist/220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Processed/N2V_Denoised/16bit_scaleZ'
save_dir = os.path.join(processing_dir, 'sbdl2_processed')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

im_names = [f for f in os.listdir(processing_dir) if f.endswith('.tif') and not f.startswith('.')]

with tqdm(total=len(im_names)) as pbar:
    for im_name in im_names:
        im_base = im_name.split('.')[0]
        im = imread(os.path.join(processing_dir, im_name))
        sbdl2 = filters.difference_of_gaussians(im, low_sigma=2, high_sigma=128)
        # save as 32 bit float
        imwrite(os.path.join(save_dir, f'{im_base}_sbdl2.tif'), sbdl2.astype('float32'), imagej=True, metadata={'axes': 'ZYX'})
        pbar.update(1)

# remove a label and re-save

In [ ]:
from tifffile import imread, imwrite
import numpy as np
import os

offending_label_number = 63
base = '/Volumes/bigData/wholeMount_volDist/220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Processed/N2V_Denoised/U-Net_3D/' 
name = '220624_Fix_Flvw_Emb_PI_Utr647_E02-10X-Z01_downsample_log_Crop_46-98_Target.tif' 
masks = imread(os.path.join(base, name))
base_name = name.split('.')[0]
masks[masks == offending_label_number] = 0
imwrite(os.path.join(base, base_name + '_corr.tif'), masks, imagej=True, metadata={'axes': 'ZYX'})


# Turn instance segmentation into semantic segmentation

In [ ]:
from tifffile import imread, imwrite
from scipy import ndimage
import os
import numpy as np
from tqdm import tqdm

base_dir = '/Users/bementmbp/Desktop/spindle_UNet/Masks' 
mask_dir = os.path.join(base_dir, 'Target')
#source_dir = os.path.join(base_dir, 'Source')
mask_names = [f for f in os.listdir(mask_dir) if f.endswith('.tif') and not f.startswith('.')]

save_dir = '/Users/bementmbp/Desktop/spindle_UNet/Masks/binary'
mask_save_dir = os.path.join(save_dir, 'Target')
#source_save_dir = os.path.join(save_dir, 'Source')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(mask_save_dir):
    os.makedirs(mask_save_dir)
#if not os.path.exists(source_save_dir):
#    os.makedirs(source_save_dir)

for mask_name in tqdm(mask_names):
    sem = imread(os.path.join(mask_dir, mask_name))
    labels = np.unique(sem)
    binary = np.zeros(sem.shape)
    for label in labels:
        if label == 0:
            continue
        else:
            mask = sem == label
            mask = ndimage.binary_erosion(mask, iterations = 2)
            binary[mask] = 1
    imwrite(os.path.join(mask_save_dir, mask_name), binary.astype('uint8'), imagej=True, metadata={'axes': 'ZYX'})
    mask_base = mask_name.split('Target.tif')[0]
    source_name = mask_base + 'Source.tif'
    #source = imread(os.path.join(source_dir, source_name))
    #imwrite(os.path.join(source_save_dir, source_name), source[2:-2,2:-2,2:-2].astype('uint16'), imagej=True, metadata={'axes': 'ZYX'})

# crop and save

In [ ]:
from tifffile import imread, imwrite
from scipy import ndimage
import os
import numpy as np
from tqdm import tqdm

base_dir = '/Users/bementmbp/Desktop/spindle_UNet/Masks' 
mask_names = [f for f in os.listdir(base_dir) if f.endswith('.tif') and not f.startswith('.')]

save_dir = '/Users/bementmbp/Desktop/spindle_UNet/Masks_cropped'

for mask_name in tqdm(mask_names):
    im = imread(os.path.join(mask_dir, mask_name))[2:-2,2:-2,2:-2]
    imwrite(os.path.join(save_dir, mask_name), binary[2:-2,2:-2,2:-2].astype('uint8'), imagej=True, metadata={'axes': 'ZYX'})
    mask_base = mask_name.split('Target.tif')[0]
    source_name = mask_base + 'Source.tif'
    #source = imread(os.path.join(source_dir, source_name))
    #imwrite(os.path.join(source_save_dir, source_name), source[2:-2,2:-2,2:-2].astype('uint16'), imagej=True, metadata={'axes': 'ZYX'})

# turn instance into multiclass semantic segmentation

In [ ]:
from tifffile import imread, imwrite
from scipy import ndimage
import os
import numpy as np
from tqdm import tqdm

# source dirs and names
base_dir = '/Volumes/bigData/wholeMount_volDist/U-Net_3D_Training' 
mask_dir = os.path.join(base_dir, 'Target')
source_dir = os.path.join(base_dir, 'Source')
mask_names = [f for f in os.listdir(mask_dir) if f.endswith('.tif') and not f.startswith('.')]

# dest dirs and names
save_dir = '/Volumes/bigData/wholeMount_volDist/U-Net_3D_Training_multiclass_8bit_1'
mask_save_dir = os.path.join(save_dir, 'Target')
source_save_dir = os.path.join(save_dir, 'Source')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(mask_save_dir):
    os.makedirs(mask_save_dir)
if not os.path.exists(source_save_dir):
    os.makedirs(source_save_dir)

# make the changes. background pixels are set to 0, edges are 1, cytoplasm is 2
for mask_name in tqdm(mask_names):
    instance_labels = imread(os.path.join(mask_dir, mask_name))
    labels = np.unique(instance_labels)
    binary = np.zeros(instance_labels.shape)
    for label in labels:
        if label == 0:
            continue
        else:
            mask = instance_labels == label
            eroded = ndimage.binary_erosion(mask, iterations = 2)
            binary[mask] = 1
            binary[eroded] = 2
    imwrite(os.path.join(mask_save_dir, mask_name), binary[2:-2,2:-2,2:-2].astype('uint8'), imagej=True, metadata={'axes': 'ZYX'})
    mask_base = mask_name.split('Target.tif')[0]
    source_name = mask_base + 'Source.tif'
    source = imread(os.path.join(source_dir, source_name))
    imwrite(os.path.join(source_save_dir, source_name), source[2:-2,2:-2,2:-2].astype('uint16'), imagej=True, metadata={'axes': 'ZYX'})

# Convert file to 8-bit

In [ ]:
from tifffile import imread, imwrite
import os
from tqdm import tqdm

base_dir = '/Volumes/bigData/wholeMount_volDist/U-Net_3D_Training_Binary_8bit' 
mask_dir = os.path.join(base_dir, 'Target')
source_dir = os.path.join(base_dir, 'Source')
mask_names = [f for f in os.listdir(mask_dir) if f.endswith('.tif') and not f.startswith('.')]

for mask_name in tqdm(mask_names):
    mask = imread(os.path.join(mask_dir, mask_name))
    imwrite(os.path.join(mask_dir, mask_name), mask.astype('uint8'), imagej=True, metadata={'axes': 'ZYX'})
    mask_base = mask_name.split('Target.tif')[0]
    source_name = mask_base + 'Source.tif'
    source = imread(os.path.join(source_dir, source_name))
    imwrite(os.path.join(source_dir, source_name), source[1:-1].astype('uint8'), imagej=True, metadata={'axes': 'ZYX'})

# Manual annotation of cell cubes

In [ ]:
import os
import numpy as np
import napari
from tifffile import imread, imwrite
from skimage.morphology import label
from skimage.filters import threshold_otsu

%gui qt 
viewer=napari.Viewer()

In [ ]:
base = '/Users/bementmbp/Desktop/test_dir/727' 
name = 'curr_tub_cube.tif' 

im = imread(os.path.join(base, name))
viewer.add_image(im, name='im', blending='additive')
thresh = threshold_otsu(im)
print(thresh)
mask = label(im > thresh)
#viewer.add_labels(mask, name='mask', blending='additive')

labels_to_remove = [1,2,3]
for l in labels_to_remove:
    mask[mask == l] = 0 

viewer.add_labels(mask, name='mask', blending='additive')

viewer.layers['mask'].save(os.path.join(base, 'thresh_mask.tif'))

In [6]:
import os
import numpy as np
import napari
from tifffile import imread, imwrite
from skimage.morphology import label
from skimage.filters import threshold_otsu

deletionviewer=napari.Viewer()

/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/jupyter_client/threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "/Users/bementmbp/opt/anaconda3/envs/napari_apo

In [7]:
maskpath = '/Users/bementmbp/Desktop/Scripts/volume-distribution/classifier/good_label_folders/82/thresh_mask.tif'

mask = imread(maskpath)

mask = np.zeros(mask.shape).astype('uint8')
deletionviewer.add_labels(mask, name='mask', blending='additive')
deletionviewer.layers['mask'].save(maskpath)

/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/threading.py:239: RuntimeWarning: coroutine 'get_msg' was never awaited
  self._release_save = lock._release_save


['/Users/bementmbp/Desktop/Scripts/volume-distribution/classifier/good_label_folders/82/thresh_mask.tif']

# automatically export volumes to images and labels directories

In [ ]:
import os
import numpy as np
import napari
from tifffile import imread, imwrite
from skimage.morphology import label
from skimage.filters import threshold_otsu

%gui qt 
viewer=napari.Viewer()

In [25]:
main_dir = '/Users/bementmbp/Desktop/autosave2/Exp_E10' 
save_dir = '/Users/bementmbp/Desktop/iter_three'   

good_examples = [104, 123, 142, 191]

for ex in good_examples:
    ex_path = os.path.join(main_dir, str(ex))
    raw_tub = imread(os.path.join(ex_path, 'curr_tub_raw_cube.tif'))
    filt_tub = imread(os.path.join(ex_path, 'curr_tub_cube.tif'))
    mask = imread(os.path.join(ex_path, 'thresh_mask.tif'))

    image_save_dir = os.path.join(save_dir, 'Images')
    if not os.path.exists(image_save_dir):
        os.makedirs(image_save_dir)
    
    mask_save_dir = os.path.join(save_dir, 'Masks')
    if not os.path.exists(mask_save_dir):
        os.makedirs(mask_save_dir)
    
    imwrite(os.path.join(image_save_dir, f'curr_tub_raw_cube_{ex}.tif'), raw_tub)
    imwrite(os.path.join(image_save_dir, f'curr_tub_cube_{ex}.tif'), filt_tub)

    label_nums = np.unique(mask)
    binary = np.zeros(mask.shape, dtype='uint8')

    for label in label_nums:
        if label == 0:
            continue
        else:
            label_mask = mask == label
            binary[label_mask] = 1
    
    imwrite(os.path.join(mask_save_dir, f'curr_tub_raw_cube_{ex}.tif'), binary)
    imwrite(os.path.join(mask_save_dir, f'curr_tub_cube_{ex}.tif'), binary)

#

# main workflow below!!

# Filtering low quality segmentations:

In [11]:
import os
import numpy as np
import napari
from tifffile import imread, imwrite
from skimage import morphology
from tqdm import tqdm
from skimage.filters import threshold_multiotsu
import time
import matplotlib.pyplot as plt
from skimage.segmentation import clear_border
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage import filters
from skimage.morphology import label
from skimage.morphology import binary_erosion
from skimage.filters import threshold_otsu
from skimage.measure import regionprops
from scipy import spatial
import vg # vector-geometry and linear-algebra toolbelt

In [12]:
%gui qt 
viewer = napari.Viewer()

/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


#### Read the data and define useful functions:

#### Filter poor quality labels:

In [40]:
from skimage.morphology import label

base_dir = '/Volumes/bigData/wholeMount_volDist/220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Processed/N2V_Denoised/16bit_scaleZ_sbdl2' 

cntrl = False
emb_nums = ['06']
for emb_num in emb_nums:

    if cntrl == True:
        emb_type = 'Cntrl'
        segmentations_name = f'220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_{emb_type}_E{emb_num}-Z01_PI_16bit_scaleZ_sbdl2_16bit_seg.npy'
        dog_tub_name = f'220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_{emb_type}_E{emb_num}-Z01_Tub_16bit_scaleZ_sbdl2_16bit.tif'
        raw_tub_name = f'220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_{emb_type}_E{emb_num}-Z01_Tub_16bit_scaleZ.tif'
        pi_name = f'220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_{emb_type}_E{emb_num}-Z01_PI_16bit_scaleZ_sbdl2_16bit.tif'

        masks = np.load(os.path.join(base_dir, segmentations_name), allow_pickle=True).item()['masks']
        tub = imread(os.path.join(base_dir, dog_tub_name))
        tub_raw = imread(os.path.join(base_dir, raw_tub_name))
        pi = imread(os.path.join(base_dir, pi_name))

    if cntrl == False:
        emb_type = 'Exp'
        segmentations_name = f'220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_{emb_type}_E{emb_num}-Z01_PI_16bit_scaleZ_sbdl2_16bit_seg.npy'
        dog_tub_name = f'220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_{emb_type}_E{emb_num}-Z01_Tub_16bit_scaleZ_sbdl2_16bit.tif'
        raw_tub_name = f'220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_{emb_type}_E{emb_num}-Z01_Tub_16bit_scaleZ.tif'
        pi_name = f'220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_{emb_type}_E{emb_num}-Z01_PI_16bit_scaleZ_sbdl2_16bit.tif'

        masks = np.load(os.path.join(base_dir, segmentations_name), allow_pickle=True).item()['masks']
        tub = imread(os.path.join(base_dir, dog_tub_name))
        tub_raw = imread(os.path.join(base_dir, raw_tub_name))
        pi = imread(os.path.join(base_dir, pi_name))

    ########################################################################################################################

    minimum_size = 250000
    maximum_size = 1000000
    minimum_density = 0.21

    filtered_masks = clear_border(masks)
    filtered_masks = morphology.remove_small_objects(filtered_masks, min_size=minimum_size, connectivity=1)
    filtered_masks = remove_large_objects(filtered_masks, max_size=maximum_size)
    remaining_labels = [label for label in np.unique(filtered_masks) if label != 0]
    print('Calculating point clouds...')
    label_pcs = [return_points(masks, label_ID) for label_ID in tqdm(remaining_labels)]
    densities = [find_label_density(pc) for pc in label_pcs]

    for ind, id in enumerate(remaining_labels):
        if id in remaining_labels and densities[ind] < minimum_density:
            filtered_masks[filtered_masks == id] = 0

    ########################################################################################################################


    final_labels = [label for label in np.unique(filtered_masks) if label != 0]
    main_save_dir = '/Users/bementmbp/Desktop/autosave2' 
    results = {}

    def calculate_geometries(curr_mask_id):

        # start with a fresh slate
        wipe_layers()
        results[curr_mask_id] = []
        curr_mask = masks == curr_mask_id

        # get the coordinate of the bounding cube for the current mask ID. Apply it to the labels and images
        cube_dims = get_cube(masks, curr_mask_id)
        cubed_label = apply_cube(curr_mask, cube_dims)
        cubed_tub = apply_cube(tub, cube_dims)
        cubed_tub_raw = apply_cube(tub_raw, cube_dims)
        cubed_PI = apply_cube(pi, cube_dims)

        # erode the mask to eliminate some cortical signal
        eroded_mask = binary_erosion(cubed_label, footprint=np.ones((3, 3, 3)))
        for i in range(10):
            eroded_mask = binary_erosion(eroded_mask)

        # get the tubulin signal from the remaining region and define an Otsu threshold
        remaining_tub = np.zeros(shape=cubed_label.shape)
        remaining_tub[eroded_mask] = cubed_tub[eroded_mask]
        remaining_vals = cubed_tub[eroded_mask].ravel()
        thresh_val = threshold_otsu(remaining_vals)
        thresh_mask = label(remaining_tub > thresh_val)

        # filter and labels smaller or larger than the mimum and maximum expected label sizes
        min_thrsh_size = 500
        max_thrsh_size = 5000
        num_tub_labels_b4_filter = [label for label in np.unique(thresh_mask) if label != 0]

        if len(num_tub_labels_b4_filter) == 0:
            # no filtering required if we already have no labels
            print(f'no spindles detected in label {curr_mask_id}')
            return

        if len(num_tub_labels_b4_filter) == 1:
            # remove_small_objects will complain about this, so let's just pass it by remove_large_objects
            thresh_mask = remove_large_objects(thresh_mask, max_size=max_thrsh_size)

        if len(num_tub_labels_b4_filter) > 1:
            # here it's likely that we have both small and large objects coontaminating
            thresh_mask = morphology.remove_small_objects(thresh_mask, min_size=min_thrsh_size, connectivity=1)
            thresh_mask = remove_large_objects(thresh_mask, max_size=max_thrsh_size)

        # get the number of labels after filtering
        remaining_labels = [label for label in np.unique(thresh_mask) if label != 0]
        if len(remaining_labels) > 1:
            print('more than one region remaining!')
            return

        if len(remaining_labels) == 0:
            print(f'no spindles detected in mask {curr_mask_id}')
            return

        # get the spindle coordinates and centroid
        spindle_label_ID = remaining_labels[0]
        '''
        record the spindle properties so we can make sure we're not analyzing any crazy shapes
        '''
        spindle_coords = np.column_stack(np.where(thresh_mask == spindle_label_ID))
        spindle_centroid = spindle_coords.mean(axis=0)
        spindle_long_vect, spindle_long_line = get_long_axis(thresh_mask)

        # get the mask coordinates, centroid, and long axis
        mask_coords = np.column_stack(np.where(cubed_label == True))
        cell_centroid = mask_coords.mean(axis=0)
        cell_long_vect, cell_long_line = get_long_axis(cubed_label)

        # get the distance between the centroids, and the angles between the long axes
        dist = spatial.distance.euclidean(cell_centroid, spindle_centroid)
        ang = vg.angle(spindle_long_vect, cell_long_vect)
        results[curr_mask_id].append(dist)
        results[curr_mask_id].append(ang)

        # make a save directory for this mask
        if cntrl:
            emb_save_dir = os.path.join(main_save_dir, f'Cntrl_E{emb_num}')
        else:
            emb_save_dir = os.path.join(main_save_dir, f'Exp_E{emb_num}')
        if not os.path.exists(emb_save_dir):
            os.makedirs(emb_save_dir)
        
        save_dir = os.path.join(emb_save_dir, f'{curr_mask_id}')
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)

        # populate the viewer 
        viewer.add_labels(eroded_mask, name='eroded_mask', blending='additive', visible=False)
        viewer.add_labels(cubed_label, name='curr_mask_cube', blending='additive')
        viewer.add_image(cubed_tub, name='curr_tub_cube', blending='additive', visible=False)
        viewer.add_image(cubed_tub_raw, name='curr_tub_raw_cube', blending='additive', visible=False)
        viewer.add_image(cubed_PI, name='curr_PI_cube', blending='additive', visible=False)
        viewer.add_labels(thresh_mask, name='thresh_mask', blending='additive')
        viewer.add_points(cell_centroid, name='spindle centroid', face_color='magenta', blending='additive')
        viewer.add_points(spindle_centroid, name='spindle centroid', face_color='green', blending='additive')
        viewer.add_shapes(cell_long_line, shape_type='line', name='cell long axis', edge_color='red', blending='additive')
        viewer.add_shapes(spindle_long_line, shape_type='line', name='spindle long axis', edge_color='blue', blending='additive')

        images_and_layers = ['curr_mask_cube',
                            'curr_tub_cube',
                            'curr_tub_raw_cube',
                            'curr_PI_cube',
                            'eroded_mask',
                            'thresh_mask']

        # save the tif compatible layers as tifs
        for item in images_and_layers:
            viewer.layers[item].save(os.path.join(save_dir, item + '.tif'))

        # save the arrays as txt files
        np.savetxt(os.path.join(save_dir, 'spindle_centroid.txt'), spindle_centroid)
        np.savetxt(os.path.join(save_dir, 'cell_centroid.txt'), cell_centroid)
        np.savetxt(os.path.join(save_dir, 'spindle_long_axis.txt'), spindle_long_line)

    for curr_mask_id in final_labels:
        calculate_geometries(curr_mask_id)
    
    print(f'finished with {emb_num}')

Calculating point clouds...


100%|██████████| 84/84 [00:31<00:00,  2.67it/s]


no spindles detected in mask 81
no spindles detected in mask 83
no spindles detected in mask 97
no spindles detected in mask 102
no spindles detected in mask 107
no spindles detected in mask 120
no spindles detected in mask 122
no spindles detected in mask 125
no spindles detected in mask 126
no spindles detected in mask 127
more than one region remaining!
no spindles detected in mask 149
no spindles detected in mask 156
no spindles detected in mask 163
no spindles detected in mask 164
no spindles detected in mask 167
no spindles detected in mask 168
no spindles detected in mask 180
no spindles detected in mask 184
no spindles detected in mask 190
no spindles detected in mask 192
no spindles detected in mask 194
no spindles detected in mask 199
no spindles detected in mask 200
no spindles detected in mask 203
no spindles detected in mask 205
no spindles detected in mask 207
no spindles detected in mask 210
no spindles detected in mask 211
no spindles detected in mask 221
no spindles de

727 is bad

getting an error with one of the masks in Exp E01

In [32]:

base_dir = '/Volumes/bigData/wholeMount_volDist/220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Processed/N2V_Denoised/16bit_scaleZ_sbdl2' 

segmentations_name = '220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Exp_E06-Z01_PI_16bit_scaleZ_sbdl2_16bit_seg.npy'
dog_tub_name = '220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Exp_E06-Z01_Tub_16bit_scaleZ_sbdl2_16bit.tif'
raw_tub_name = '220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Exp_E06-Z01_Tub_16bit_scaleZ.tif'
pi_name = '220712_Fix_Emb_Flvw_Chn1GAP_PI_aTub647_Exp_E06-Z01_PI_16bit_scaleZ_sbdl2_16bit.tif'

masks = np.load(os.path.join(base_dir, segmentations_name), allow_pickle=True).item()['masks']
tub = imread(os.path.join(base_dir, dog_tub_name))
tub_raw = imread(os.path.join(base_dir, raw_tub_name))
pi = imread(os.path.join(base_dir, pi_name))

print(masks.shape)
print(tub.shape)
print(tub_raw.shape)
print(pi.shape)

(396, 640, 640)
(396, 640, 640)
(396, 640, 640)
(396, 640, 640)


In [33]:
viewer = napari.Viewer()

/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [34]:
minimum_size = 250000
maximum_size = 1000000
minimum_density = 0.21

filtered_masks = clear_border(masks)
filtered_masks = morphology.remove_small_objects(filtered_masks, min_size=minimum_size, connectivity=1)
filtered_masks = remove_large_objects(filtered_masks, max_size=maximum_size)
remaining_labels = [label for label in np.unique(filtered_masks) if label != 0]
print('Calculating point clouds...')
label_pcs = [return_points(masks, label_ID) for label_ID in tqdm(remaining_labels)]
densities = [find_label_density(pc) for pc in label_pcs]

for ind, id in enumerate(remaining_labels):
    if id in remaining_labels and densities[ind] < minimum_density:
        filtered_masks[filtered_masks == id] = 0

/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/zmq/_future.py:466: RuntimeWarning: coroutine 'get_msg' was never awaited
  if not self._shadow_sock.get(EVENTS) & POLLIN:


Calculating point clouds...


100%|██████████| 84/84 [00:30<00:00,  2.77it/s]


In [35]:
final_labels = [label for label in np.unique(filtered_masks) if label != 0]
main_save_dir = '/Users/bementmbp/Desktop/autosave2' 
results = {}

In [39]:

def calculate_geometries(curr_mask_id):

    print(curr_mask_id)

    # start with a fresh slate
    wipe_layers()
    results[curr_mask_id] = []
    curr_mask = masks == curr_mask_id

    # get the coordinate of the bounding cube for the current mask ID. Apply it to the labels and images
    cube_dims = get_cube(masks, curr_mask_id)
    cubed_label = apply_cube(curr_mask, cube_dims)
    cubed_tub = apply_cube(tub, cube_dims)
    cubed_tub_raw = apply_cube(tub_raw, cube_dims)
    cubed_PI = apply_cube(pi, cube_dims)

    # erode the mask to eliminate some cortical signal
    eroded_mask = binary_erosion(cubed_label, footprint=np.ones((3, 3, 3)))
    for i in range(10):
        eroded_mask = binary_erosion(eroded_mask)

    # get the tubulin signal from the remaining region and define an Otsu threshold
    remaining_tub = np.zeros(shape=cubed_label.shape)
    remaining_tub[eroded_mask] = cubed_tub[eroded_mask]
    remaining_vals = cubed_tub[eroded_mask].ravel()
    thresh_val = threshold_otsu(remaining_vals)
    print(thresh_val)
    blah = remaining_tub > thresh_val
    print(blah.shape)
    thresh_mask = label()

    # filter and labels smaller or larger than the mimum and maximum expected label sizes
    min_thrsh_size = 500
    max_thrsh_size = 5000
    num_tub_labels_b4_filter = [label for label in np.unique(thresh_mask) if label != 0]

    if len(num_tub_labels_b4_filter) == 0:
        # no filtering required if we already have no labels
        print(f'no spindles detected in label {curr_mask_id}')
        return

    if len(num_tub_labels_b4_filter) == 1:
        # remove_small_objects will complain about this, so let's just pass it by remove_large_objects
        thresh_mask = remove_large_objects(thresh_mask, max_size=max_thrsh_size)

    if len(num_tub_labels_b4_filter) > 1:
        # here it's likely that we have both small and large objects coontaminating
        thresh_mask = morphology.remove_small_objects(thresh_mask, min_size=min_thrsh_size, connectivity=1)
        thresh_mask = remove_large_objects(thresh_mask, max_size=max_thrsh_size)

    # get the number of labels after filtering
    remaining_labels = [label for label in np.unique(thresh_mask) if label != 0]
    if len(remaining_labels) > 1:
        print('more than one region remaining!')
        return

    if len(remaining_labels) == 0:
        print(f'no spindles detected in mask {curr_mask_id}')
        return

    # get the spindle coordinates and centroid
    spindle_label_ID = remaining_labels[0]
    '''
    record the spindle properties so we can make sure we're not analyzing any crazy shapes
    '''
    spindle_coords = np.column_stack(np.where(thresh_mask == spindle_label_ID))
    spindle_centroid = spindle_coords.mean(axis=0)
    spindle_long_vect, spindle_long_line = get_long_axis(thresh_mask)

    # get the mask coordinates, centroid, and long axis
    mask_coords = np.column_stack(np.where(cubed_label == True))
    cell_centroid = mask_coords.mean(axis=0)
    cell_long_vect, cell_long_line = get_long_axis(cubed_label)

    # get the distance between the centroids, and the angles between the long axes
    dist = spatial.distance.euclidean(cell_centroid, spindle_centroid)
    ang = vg.angle(spindle_long_vect, cell_long_vect)
    results[curr_mask_id].append(dist)
    results[curr_mask_id].append(ang)

    # make a save directory for this mask
    if cntrl:
        emb_save_dir = os.path.join(main_save_dir, f'Cntrl_E{emb_num}')
    else:
        emb_save_dir = os.path.join(main_save_dir, f'Exp_E{emb_num}')
    if not os.path.exists(emb_save_dir):
        os.makedirs(emb_save_dir)
    
    save_dir = os.path.join(emb_save_dir, f'{curr_mask_id}')
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    # populate the viewer 
    viewer.add_labels(eroded_mask, name='eroded_mask', blending='additive', visible=False)
    viewer.add_labels(cubed_label, name='curr_mask_cube', blending='additive')
    viewer.add_image(cubed_tub, name='curr_tub_cube', blending='additive', visible=False)
    viewer.add_image(cubed_tub_raw, name='curr_tub_raw_cube', blending='additive', visible=False)
    viewer.add_image(cubed_PI, name='curr_PI_cube', blending='additive', visible=False)
    viewer.add_labels(thresh_mask, name='thresh_mask', blending='additive')
    viewer.add_points(cell_centroid, name='spindle centroid', face_color='magenta', blending='additive')
    viewer.add_points(spindle_centroid, name='spindle centroid', face_color='green', blending='additive')
    viewer.add_shapes(cell_long_line, shape_type='line', name='cell long axis', edge_color='red', blending='additive')
    viewer.add_shapes(spindle_long_line, shape_type='line', name='spindle long axis', edge_color='blue', blending='additive')

    images_and_layers = ['curr_mask_cube',
                        'curr_tub_cube',
                        'curr_tub_raw_cube',
                        'curr_PI_cube',
                        'eroded_mask',
                        'thresh_mask']

    # save the tif compatible layers as tifs
    for item in images_and_layers:
        viewer.layers[item].save(os.path.join(save_dir, item + '.tif'))

    # save the arrays as txt files
    np.savetxt(os.path.join(save_dir, 'spindle_centroid.txt'), spindle_centroid)
    np.savetxt(os.path.join(save_dir, 'cell_centroid.txt'), cell_centroid)
    np.savetxt(os.path.join(save_dir, 'spindle_long_axis.txt'), spindle_long_line)

for curr_mask_id in final_labels:
    calculate_geometries(curr_mask_id)

print(f'finished with {emb_num}')


67
5279
(148, 92, 107)


TypeError: 'numpy.int32' object is not callable

In [41]:
methodsViewer = napari.Viewer()

/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/jupyter_client/threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "/Users/bementmbp/opt/anaconda3/envs/napari_apo

In [42]:
from skimage import data
import elasticdeform

mask = imread('/Users/bementmbp/Desktop/autosave2/Exp_E10/169/thresh_mask.tif')
tub = imread('/Users/bementmbp/Desktop/autosave2/Exp_E10/169/curr_tub_raw_cube.tif')
methodsViewer.add_labels(mask, name='mask', blending='additive')
methodsViewer.add_image(tub, name='tub', blending='additive')

sigma_= 2 #deformation parameter
points_= 2 #deformation parameter

[defmask, deftub] = elasticdeform.deform_random_grid([mask, tub], sigma=sigma_, points=points_)
methodsViewer.add_labels(defmask, name='defmask', blending='additive')
methodsViewer.add_image(deftub, name='deftub', blending='additive')



/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/concurrent/futures/thread.py:162: RuntimeWarning: coroutine 'get_msg' was never awaited
  with self._shutdown_lock, _global_shutdown_lock:


<Image layer 'deftub' at 0x7ff1da754940>

In [ ]:
propsviewer = napari.Viewer()

In [ ]:
from skimage.measure import label, regionprops, regionprops_table
import pandas as pd

test_dir = '/Users/bementmbp/Desktop/autosave2/Exp_E02/46' 
mask_cube_name = 'curr_mask_cube.tif'
tub_cube_name = 'curr_tub_cube.tif'


my_labels = label(imread('/Users/bementmbp/Desktop/iter_4/Masks/curr_tub_raw_cube_46.tif'))

regions = regionprops(my_labels)
props = regionprops_table(my_labels, properties=('area',
                                                 'axis_major_length',
                                                 'axis_minor_length'))
props_df = pd.DataFrame(props)
props_df